# Load required libraries

In [2]:
import pandas as pd
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# Download the datasets from private S3 bucket

In [4]:
!aws s3 cp 's3://ads508-team4-master/df_psych.csv' ./data/

download: s3://ads508-team4-master/df_psych.csv to data/df_psych.csv


In [5]:
import csv

df_psych = pd.read_csv(
    "./data/df_psych.csv",
    delimiter=",",
    quoting=csv.QUOTE_NONE,
)
df_psych = df_psych.iloc[:,1:]
df_psych.head(100)

,user_id,platform_x,level_1,level_2,level_3,confidence_score,country_code,asset_id,minutes_viewed,showtype,genre,running_minutes,source_language,season_id,series_id,studio_id,minutes_under_2
0,8.589935e+10,web-embed,Psychographics,Movies Lovers,Horror Movies Fans,0.07,ID,10377,1,Movies,Horror,87,Indonesian,NaN,NaN,350.0,True
1,8.589935e+10,web-embed,Psychographics,Movies Lovers,Indonesian Movies Fans,0.03,ID,10377,1,Movies,Horror,87,Indonesian,NaN,NaN,350.0,True
2,2.576980e+10,android,Psychographics,Movies Lovers,Romance Movies Fans,0.52,ID,10377,3,Movies,Horror,87,Indonesian,NaN,NaN,350.0,False
3,2.576980e+10,android,Psychographics,TV Lovers,Kids TV Fans,0.61,ID,10377,3,Movies,Horror,87,Indonesian,NaN,NaN,350.0,False
4,2.576980e+10,android,Psychographics,TV Lovers,Drama TV Fans,0.60,ID,10377,3,Movies,Horror,87,Indonesian,NaN,NaN,350.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7.700000e+01,android,Psychographics,Movies Lovers,Korean Movies Fans,0.54,ID,10377,12,Movies,Horror,87,Indonesian,NaN,NaN,350.0,False
96,7.700000e+01,android,Psychographics,Movies Lovers,English Movies Fans,0.46,ID,10377,12,Movies,Horror,87,Indonesian,NaN,NaN,350.0,False
97,7.700000e+01,android,Psychographics,TV Lovers,English TV Fans,0.40,ID,10377,12,Movies,Horror,87,Indonesian,NaN,NaN,350.0,False
98,6.871948e+10,android,Psychographics,TV Lovers,English TV Fans,0.29,ID,10377,3,Movies,Horror,87,Indonesian,NaN,NaN,350.0,False
